<a href="https://colab.research.google.com/github/darshan-k3/Low-Complexity-Deepfake-Detection-Model/blob/main/Creation_of_Benchmark_Dataset_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This script demonstrates how the benchmark dataset was created.
# @author 21004275

In [ ]:
#@title 0. Creation of Benchmark Dataset
import os
import random
from collections import defaultdict

# Define base directories
base_dirs = {
    '/content/drive/MyDrive/FakeAVCeleb_v1.2_full/FakeAVCeleb_v1.2/RealVideo-RealAudio': (0, 0),
    '/content/drive/MyDrive/FakeAVCeleb_v1.2_full/FakeAVCeleb_v1.2/RealVideo-FakeAudio': (0, 1),
    '/content/drive/MyDrive/FakeAVCeleb_v1.2_full/FakeAVCeleb_v1.2/FakeVideo-RealAudio': (1, 0),
    '/content/drive/MyDrive/FakeAVCeleb_v1.2_full/FakeAVCeleb_v1.2/FakeVideo-FakeAudio': (1, 1)
}

ethnicities = ['Caucasian (European)', 'Asian (East)', 'Asian (South)', 'African', 'Caucasian (American)']
sexes = ['men', 'women']

# /content/drive/MyDrive/FakeAVCeleb_v1.2_full/FakeAVCeleb_v1.2/RealVideo-RealAudio/Caucasian (European)/men/id03205
def collect_representative_files(base_dir):
    video_files = []
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.mp4'):
                video_files.append(os.path.join(root, file))
    final_paths = []
    for eth in ethnicities:
        for sex in sexes:
            # Filter and sample in one step
            filtered_paths = [path for path in video_files if eth == path.split('/')[-4] and sex == path.split('/')[-3]]
            # if len(filtered_paths) < 20:
            # print(f"Warning: Only found {len(filtered_paths)} videos for {eth}, {sex} in {base_dir}")
            final_paths.extend(random.sample(filtered_paths, 20))
            print(f'{eth}, {sex} with final paths length {len(final_paths)}')

    print(len(final_paths))
    return final_paths

# Prepare the training data
# for base_dir, label in base_dirs.items():
#     random_files = collect_representative_files(base_dir)
#     training_path_dict.update({file: label for file in random_files})

training_path_dict = defaultdict()
test_list = []
for base_dir, label in base_dirs.items():
    random_files = collect_representative_files(base_dir)
    # Update keys to be tuples of (file_path, label), ensuring uniqueness
    for file in random_files:
        test_list.append(file)
        # This creates a unique key for each file by combining its path and the label
        training_path_dict[(file, label)] = label
# Now, training_path_dict contains the paths and labels


In [ ]:
len(training_path_dict)

800

In [ ]:
#@title 0.1. Identify duplicates in the list
def identify_duplicates(input_list):
    seen = set()
    duplicates = set()
    for item in input_list:
        # If the item is already in seen, add to duplicates
        if item in seen:
            duplicates.add(item)
        else:
            seen.add(item)
    return list(duplicates)
lst = identify_duplicates(training_path_dict)

lst

[]

In [ ]:
#@title 0.2. Copy out to relevant folders
import shutil
import os

real_real_dir = os.path.join('benchmark_dataset', 'RealVideo-RealAudio')
real_fake_dir = os.path.join('benchmark_dataset', 'RealVideo-FakeAudio')
fake_real_dir = os.path.join('benchmark_dataset', 'FakeVideo-RealAudio')
fake_fake_dir = os.path.join('benchmark_dataset', 'FakeVideo-FakeAudio')

os.makedirs(real_real_dir, exist_ok=True)
os.makedirs(real_fake_dir, exist_ok=True)
os.makedirs(fake_real_dir, exist_ok=True)
os.makedirs(fake_fake_dir, exist_ok=True)


for path, label in training_path_dict.items():
  if label == (0, 0): # real video real audio
    shutil.copy(path[0], os.path.join(real_real_dir, "_".join(path[0].split('/')[-3:])))
  if label == (0, 1): # real video fake audio
    shutil.copy(path[0], os.path.join(real_fake_dir, "_".join(path[0].split('/')[-3:])))
  if label == (1, 0): # fake video real audio
    shutil.copy(path[0], os.path.join(fake_real_dir, "_".join(path[0].split('/')[-3:])))
  if label == (1, 1): # fake video fake audio
    shutil.copy(path[0], os.path.join(fake_fake_dir, "_".join(path[0].split('/')[-3:])))


In [ ]:
#@title 0.3. Check that folder was properly created.
len(os.listdir(os.path.join('benchmark_dataset', 'FakeVideo-FakeAudio')))

200

In [ ]:
#@title 0.4. Zip Benchmark Dataset
# !zip -r benchmark_dataset.zip benchmark_dataset

Note that because the benchmark dataset has been created, these steps are only run once.